# CFB Predictions

## Data Source:

collegefootballdata.com

### Query Parameters

WeekX-Games.csv: "Games and Results", Week = X, Division = fbs 

Team-EPA.csv: "Predicted Points Added by Team", Exclude Garbage Time

Player-PPA.csv: "Player Predicted Points Added by Season", Minimum of 5 snaps (Threshold), Exclude Garbage Time

Player-Snaps.csv: "Player Usage metrics by season", Exclude Garbage Time

Team-Plays.csv: "Advanced Team Metrics by season", Exclude Garbage Time, Add Games manually

In [50]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

week3 = pd.read_csv("Week3-Games.csv")
week3.head()

,Id,Season,Week,Season Type,Neutral Site,Conference Game,Venue Id,Venue,Home Id,Home Team,Home Conference,Home Division,Home Pregame Elo,Away Id,Away Team,Away Conference,Away Division,Away Pregame Elo
0,401525499,2023,3,regular,False,False,3948,Hard Rock Stadium,2390,Miami,ACC,fbs,1693,2065,Bethune-Cookman,SWAC,fcs,NaN
1,401531428,2023,3,regular,False,True,3805,Liberty Bowl Memorial Stadium,235,Memphis,American Athletic,fbs,1564,2426,Navy,American Athletic,fbs,1320.0
2,401531396,2023,3,regular,False,False,3604,Alamodome,2636,UT San Antonio,American Athletic,fbs,1514,349,Army,FBS Independents,fbs,1530.0
3,401520231,2023,3,regular,False,False,3665,Maryland Stadium,120,Maryland,Big Ten,fbs,1522,258,Virginia,ACC,fbs,1424.0
4,401532581,2023,3,regular,False,True,3713,Falcon Stadium,2005,Air Force,Mountain West,fbs,1650,328,Utah State,Mountain West,fbs,1553.0


In [51]:
def elo_win_prob(home_elo, away_elo):
    elo_diff = home_elo - away_elo
    s = 400 #Scaling factor
    return 1/(1+10**(-elo_diff/s))

In [54]:
week3 = week3.dropna().reset_index(drop=True) #Apparently I'm running an old pandas version
week3["Home Win Prob"] = elo_win_prob(week3["Home Pregame Elo"], week3["Away Pregame Elo"])
week3.head()

,Id,Season,Week,Season Type,Neutral Site,Conference Game,Venue Id,Venue,Home Id,Home Team,Home Conference,Home Division,Home Pregame Elo,Away Id,Away Team,Away Conference,Away Division,Away Pregame Elo,Home Win Prob
0,401531428,2023,3,regular,False,True,3805,Liberty Bowl Memorial Stadium,235,Memphis,American Athletic,fbs,1564,2426,Navy,American Athletic,fbs,1320.0,0.802909
1,401531396,2023,3,regular,False,False,3604,Alamodome,2636,UT San Antonio,American Athletic,fbs,1514,349,Army,FBS Independents,fbs,1530.0,0.476990
2,401520231,2023,3,regular,False,False,3665,Maryland Stadium,120,Maryland,Big Ten,fbs,1522,258,Virginia,ACC,fbs,1424.0,0.637408
3,401532581,2023,3,regular,False,True,3713,Falcon Stadium,2005,Air Force,Mountain West,fbs,1650,328,Utah State,Mountain West,fbs,1553.0,0.636077
4,401520249,2023,3,regular,False,False,347,Camp Randall Stadium,275,Wisconsin,Big Ten,fbs,1753,290,Georgia Southern,Sun Belt,fbs,1414.0,0.875604


In [55]:
team_epa = pd.read_csv("Team-EPA.csv")
team_epa.head()

,Season,Conference,Team,Offense Overall,Offense Passing,Offense Rushing,Offense Cumulative Total,Offense Cumulative Passing,Offense Cumulative Rushing,Defense Overall,Defense Passing,Defense Rushing,Defense Cumulative Total,Defense Cumulative Passing,Defense Cumulative Rushing
0,2023,Mountain West,Air Force,0.384590,0.615962,0.365038,59.226886,7.391548,51.835338,-0.054965,-0.025450,-0.002719,-5.826242,-1.348839,-0.141381
1,2023,Mid-American,Akron,-0.081077,0.155293,-0.117813,-8.107725,8.851695,-4.594707,-0.047635,0.159650,-0.292707,-6.668935,13.091328,-16.684273
2,2023,SEC,Alabama,0.336633,0.439349,0.239731,34.673222,21.967475,12.705748,0.173282,0.300046,0.059887,19.234341,18.602860,2.874580
3,2023,Sun Belt,Appalachian State,0.340924,0.559091,0.218142,51.138629,36.340922,18.323935,0.306235,0.232721,0.444114,44.097843,18.152232,28.423305
4,2023,Pac-12,Arizona,0.350924,0.612474,0.193302,36.145176,34.911019,8.118678,-0.094163,0.200684,-0.091130,-9.133810,7.023943,-5.558912


In [56]:
player_ppa = pd.read_csv("Player-PPA.csv")
player_ppa.head()

,Season,Id,Name,Position,Team,Conference,CountablePlays,AveragePPA All,AveragePPA Pass,AveragePPA Rush,TotalPPA All,TotalPPA Pass,TotalPPA Rush
0,2023,4690144,BJ Alexander,WR,Florida Atlantic,American Athletic,5,-0.928,-0.928,NaN,-4.640,-4.640,NaN
1,2023,4869960,KZ Adams,RB,Georgia State,Sun Belt,8,-0.831,NaN,-0.831,-6.650,NaN,-6.650
2,2023,4912275,Christian Washington,RB,New Mexico,Mountain West,7,-0.732,-0.021,-0.850,-5.122,-0.021,-5.101
3,2023,4571305,Zak Wallace,RB,Arkansas State,Sun Belt,15,-0.591,-0.621,-0.586,-8.860,-1.243,-7.617
4,2023,4373662,Billy Bowens,WR,Boise State,Mountain West,5,-0.576,-0.576,NaN,-2.878,-2.878,NaN


In [57]:
player_snaps = pd.read_csv("Player-Snaps.csv")
player_snaps.head()

,Season,Id,Name,Position,Team,Conference,Usage Overall,Usage Pass,Usage Rush
0,2023,4360689,Tyler Shough,QB,Texas Tech,Big 12,0.7919,0.9775,0.5439
1,2023,4709977,Brayden Schager,QB,Hawai'i,Mountain West,0.7164,0.9851,0.1905
2,2023,4688380,Cameron Ward,QB,Washington State,Pac-12,0.7097,0.9667,0.3538
3,2023,4370775,Brennan Armstrong,QB,NC State,ACC,0.6933,0.9867,0.4000
4,2023,4682518,Nick Vattiato,QB,Middle Tennessee,Conference USA,0.6825,0.9733,0.2600


In [58]:
team_plays = pd.read_csv("Team-Plays.csv")
team_plays.head()

,Season,Team,Conference,Offense Plays,Games,Offense RushingPlays Rate,Offense PassingPlays Rate,Defense RushingPlays Ppa,Defense PassingPlays Ppa
0,2023,Boston College,ACC,130,2,0.553846,0.438462,0.182805,0.247820
1,2023,Clemson,ACC,145,2,0.441379,0.558621,0.088369,-0.030623
2,2023,Duke,ACC,107,2,0.532710,0.467290,0.184208,0.022914
3,2023,Florida State,ACC,117,2,0.478632,0.521368,0.114610,0.074020
4,2023,Georgia Tech,ACC,50,2,0.440000,0.560000,0.001444,0.646436


In [59]:
team_plays["Tempo"] = team_plays["Offense Plays"]/team_plays["Games"] #Tempo: Avg. Offensive plays run per game
team_plays["Pass Snaps"] = team_plays["Tempo"]*team_plays["Offense PassingPlays Rate"]
team_plays["Rush Snaps"] = team_plays["Tempo"]*team_plays["Offense RushingPlays Rate"]
team_plays.head()

,Season,Team,Conference,Offense Plays,Games,Offense RushingPlays Rate,Offense PassingPlays Rate,Defense RushingPlays Ppa,Defense PassingPlays Ppa,Tempo,Pass Snaps,Rush Snaps
0,2023,Boston College,ACC,130,2,0.553846,0.438462,0.182805,0.247820,65.0,28.5,36.0
1,2023,Clemson,ACC,145,2,0.441379,0.558621,0.088369,-0.030623,72.5,40.5,32.0
2,2023,Duke,ACC,107,2,0.532710,0.467290,0.184208,0.022914,53.5,25.0,28.5
3,2023,Florida State,ACC,117,2,0.478632,0.521368,0.114610,0.074020,58.5,30.5,28.0
4,2023,Georgia Tech,ACC,50,2,0.440000,0.560000,0.001444,0.646436,25.0,14.0,11.0


In [60]:
passes = [] #List so pandas doesn't yell at us
rushes = []
for i in range(player_snaps.shape[0]):
    pass_snaps = float(team_plays[team_plays["Team"] == player_snaps["Team"].loc[i]]["Pass Snaps"]) #Lookup team rush/pass snaps
    rush_snaps = float(team_plays[team_plays["Team"] == player_snaps["Team"].loc[i]]["Rush Snaps"]) #Float conversions cause pandas
    passes.append(pass_snaps * float(player_snaps["Usage Pass"].loc[i])) #Calculate player rush/pass snaps
    rushes.append(pass_snaps * float(player_snaps["Usage Rush"].loc[i]))
    
player_snaps["Pass Snaps"] = passes
player_snaps["Rush Snaps"] = rushes
player_snaps.head()

,Season,Id,Name,Position,Team,Conference,Usage Overall,Usage Pass,Usage Rush,Pass Snaps,Rush Snaps
0,2023,4360689,Tyler Shough,QB,Texas Tech,Big 12,0.7919,0.9775,0.5439,43.498750,24.20355
1,2023,4709977,Brayden Schager,QB,Hawai'i,Mountain West,0.7164,0.9851,0.1905,44.001133,8.50900
2,2023,4688380,Cameron Ward,QB,Washington State,Pac-12,0.7097,0.9667,0.3538,26.100900,9.55260
3,2023,4370775,Brennan Armstrong,QB,NC State,ACC,0.6933,0.9867,0.4000,34.041150,13.80000
4,2023,4682518,Nick Vattiato,QB,Middle Tennessee,Conference USA,0.6825,0.9733,0.2600,31.145600,8.32000


In [61]:
temp_snaps = player_snaps.drop(columns=["Season","Id","Position","Team","Conference","Usage Overall","Usage Pass", "Usage Rush"])
player_ppa = player_ppa.merge(temp_snaps, on="Name")

In [62]:
player_ppa = player_ppa.fillna(0) #Replace NaN with 0
player_ppa.head()

,Season,Id,Name,Position,Team,Conference,CountablePlays,AveragePPA All,AveragePPA Pass,AveragePPA Rush,TotalPPA All,TotalPPA Pass,TotalPPA Rush,Pass Snaps,Rush Snaps
0,2023,4690144,BJ Alexander,WR,Florida Atlantic,American Athletic,5,-0.928,-0.928,0.000,-4.640,-4.640,0.000,2.427600,0.000000
1,2023,4869960,KZ Adams,RB,Georgia State,Sun Belt,8,-0.831,0.000,-0.831,-6.650,0.000,-6.650,0.000000,1.500000
2,2023,4912275,Christian Washington,RB,New Mexico,Mountain West,7,-0.732,-0.021,-0.850,-5.122,-0.021,-5.101,0.497000,1.667167
3,2023,4571305,Zak Wallace,RB,Arkansas State,Sun Belt,15,-0.591,-0.621,-0.586,-8.860,-1.243,-7.617,0.498333,7.196667
4,2023,4373662,Billy Bowens,WR,Boise State,Mountain West,5,-0.576,-0.576,0.000,-2.878,-2.878,0.000,3.221900,0.000000


In [64]:
home_off = []
away_off = []
for i in range(week3.shape[0]):
    home = week3["Home Team"].loc[i]
    away = week3["Away Team"].loc[i]
    home_players = player_ppa[player_ppa["Team"] == home]
    away_players = player_ppa[player_ppa["Team"] == away]
    home_pts = home_players["Pass Snaps"]*home_players["AveragePPA Pass"] + home_players["Rush Snaps"]*home_players["AveragePPA Rush"]
    away_pts = away_players["Pass Snaps"]*away_players["AveragePPA Pass"] + away_players["Rush Snaps"]*away_players["AveragePPA Rush"]
    home_off.append(home_pts.sum())
    away_off.append(away_pts.sum())
    
week3["Raw Offense (Home)"] = home_off
week3["Raw Offense (Away)"] = away_off
week3.head()

,Id,Season,Week,Season Type,Neutral Site,Conference Game,Venue Id,Venue,Home Id,Home Team,...,Home Division,Home Pregame Elo,Away Id,Away Team,Away Conference,Away Division,Away Pregame Elo,Home Win Prob,Raw Offense (Home),Raw Offense (Away)
0,401531428,2023,3,regular,False,True,3805,Liberty Bowl Memorial Stadium,235,Memphis,...,fbs,1564,2426,Navy,American Athletic,fbs,1320.0,0.802909,32.922107,7.185691
1,401531396,2023,3,regular,False,False,3604,Alamodome,2636,UT San Antonio,...,fbs,1514,349,Army,FBS Independents,fbs,1530.0,0.476990,28.651809,28.549850
2,401520231,2023,3,regular,False,False,3665,Maryland Stadium,120,Maryland,...,fbs,1522,258,Virginia,ACC,fbs,1424.0,0.637408,46.234009,27.566080
3,401532581,2023,3,regular,False,True,3713,Falcon Stadium,2005,Air Force,...,fbs,1650,328,Utah State,Mountain West,fbs,1553.0,0.636077,4.282570,26.570763
4,401520249,2023,3,regular,False,False,347,Camp Randall Stadium,275,Wisconsin,...,fbs,1753,290,Georgia Southern,Sun Belt,fbs,1414.0,0.875604,29.619913,55.441842
